# Bootstrap Confidence Intervals experiments

We will now make simple tests in several datasets (including the simulated ones) to see how our bootstrap confidence intervals works:

In [1]:
# importing libraries
from sklearn.datasets import load_boston
import numpy as np
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from nonconformist.cp import IcpRegressor
from nonconformist.nc import NcFactory
from sklearn.model_selection import train_test_split
from scipy import stats
import pandas as pd
import os
import time
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('ggplot')
sns.set_palette("Set1")

# original directory
path_original = os.getcwd()
# importing module
from lcv.valid_pred_sets import Valid_pred_sets
from lcv.valid_pred_sets import LinearQuantileRegression
from lcv.valid_pred_sets import GradientBoostingQuantileRegression

# figure path
images_dir = "figures"

## Simulated datasets
We consider the same simulated datasets generated by Izbicki et.al (2022) to visualize our bootstrap confidence intervals in different scenarios:

In [2]:
# importing simulated datasets module
from simulation import simulation

# shuffling and splitting in train, calibration and test sets
def split(X, y, test_size = 0.4, calibrate = True, random_seed = 1250):
    X_train, X_test, y_train, y_test = train_test_split(X, y,test_size = test_size,
                                                        random_state = random_seed)
    if calibrate:
        X_train, X_calib, y_train, y_calib = train_test_split(X_train, y_train, test_size = 0.25,
                                                             random_state = random_seed)
        return {"X_train":X_train, "X_calib": X_calib, "X_test" : X_test, 
                "y_train" : y_train, "y_calib" : y_calib, "y_test": y_test}
    else:
        return{"X_train":X_train,"X_test" : X_test, 
                "y_train" : y_train,"y_test": y_test}

Testing bootstrap for several models and kind of simulated data:

In [3]:
def test_bootstrap(kind = "homoscedastic", n = 10000, d = 20, coef = 2, B = 1000,
                 random_seed = 1250, sig = 0.05, sig_b = 0.05, coverage_evaluator = "RF"):
    sim_obj = simulation(dim = d, coef = coef)
    sim_kind = getattr(sim_obj, kind)
    sim_kind(n, random_seed = random_seed)
    split_icp, split_quantile = split(sim_obj.X, sim_obj.y), split(sim_obj.X, sim_obj.y, calibrate = False)
    
    # testing first icp
    model = RandomForestRegressor(random_state = random_seed)
    nc = NcFactory.create_nc(model)
    icp = IcpRegressor(nc)
    icp.fit(split_icp["X_train"], split_icp["y_train"])
    icp.calibrate(split_icp["X_calib"], split_icp["y_calib"])
    hyp_icp = Valid_pred_sets(icp, sig, coverage_evaluator = coverage_evaluator)
    hyp_icp.fit(split_icp["X_test"], split_icp["y_test"], random_seed = random_seed)
    res_icp = hyp_icp.bootstrap_ci(B = B, sig_b = sig_b, random_seed = random_seed)
    # notifying that ICP has runned
    print("ICP runned \n")
    
    # testing gradient boosting quantile regression
    gbqr = GradientBoostingQuantileRegression(n_estimators = 200, random_state = random_seed)
    gbqr.fit(split_quantile["X_train"], split_quantile["y_train"])
    hyp_gbqr_quant = Valid_pred_sets(gbqr, sig, coverage_evaluator = coverage_evaluator)
    hyp_gbqr_quant.fit(split_quantile["X_test"], split_quantile["y_test"], random_seed = random_seed)
    res_gbqr_quant = hyp_gbqr_quant.bootstrap_ci(B = B, sig_b = sig_b, random_seed = random_seed)
    # notifying that GBQR has runned
    print("GBQR runned \n")
    
    # testing linear quantile regression
    lqr = LinearQuantileRegression(alpha = 1, solver = "highs")
    lqr.fit(split_quantile["X_train"], split_quantile["y_train"])
    hyp_lqr_quant = Valid_pred_sets(lqr, sig, coverage_evaluator = coverage_evaluator)
    hyp_lqr_quant.fit(split_quantile["X_test"], split_quantile["y_test"], random_seed = random_seed)
    res_lqr_quant = hyp_lqr_quant.bootstrap_ci(B = B, sig_b = sig_b, random_seed = random_seed)
    # notifying that LQR has runned
    print("LQR runned \n")
    print("Finished testing")

    return {"ICP":res_icp, 
            "GB Quantile Reg":res_gbqr_quant, 
            "Linear Quantile Reg":res_lqr_quant}

### Homoscedastic simulated data

We will first consider the following distribution parameters and dataset settings:
* $\beta_1 = 2$, $\sigma = 1$ and $d = 20$

In [4]:
# using test_bootstrap
# measuring time
start = time.time()
print("Homoscedastic bootstrap CI:")
boot_hom = test_bootstrap()
end = time.time() - start
print("Time Elapsed: ", end)
boot_hom

Homoscedastic bootstrap CI:
